In [ ]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import networkx as nx

In [ ]:
import pysad.pysad.collect as collect
import pysad.pysad.synthesis as synthesis
import pysad.pysad.graph as graph
from pysad.pysad.NodeInfo import SynthNodeInfo

# Creating the graph

In [ ]:
data_path = '../synthesis/'
graphname = 'Erdos-Renyi'

In [ ]:
nb_nodes = 2000
nb_edges = 10000
G = nx.gnm_random_graph(nb_nodes, nb_edges)
G.graph['name'] = graphname

In [ ]:
graph_handle = synthesis.SyntheticNetwork(G)

In [ ]:
exploration_depth = 5 # mention of mention of mention of ... up to exploration depth

In [ ]:
total_node_list, total_nodes_df, total_edges_df, node_acc = collect.spiky_ball([20], 
                                                                               graph_handle, 
                                                                               exploration_depth=exploration_depth,
                                                                               random_subset_size=0.8, node_acc=SynthNodeInfo())

In [ ]:
print('Total number of nodes in the spiky ball:',len(total_node_list))

In [ ]:
node_df, edge_df = total_nodes_df, total_edges_df
node_df = synthesis.reshape_node_data(node_df)
edge_df = synthesis.reshape_edge_data(edge_df)

## Creating the graph

In [ ]:
MIN_WEIGHT = 0
MIN_DEGREE = 2 # Minimal number of connections in the graph

Gsp = graph.graph_from_edgeslist(edge_df, MIN_WEIGHT)
#G = pysad.graph.graph_from_edgeslist(df_pop,DEGREE_MIN)
G = graph.add_edges_attributes(G,edge_df)
G = graph.add_node_attributes(G,node_df)
Gsp = graph.add_edges_attributes(Gsp,edge_df)
Gsp = graph.add_node_attributes(Gsp,node_df)

Gsp = graph.reduce_graph(Gsp,MIN_DEGREE)
Gsp = graph.handle_spikyball_neighbors(Gsp,graph_handle,remove=False, node_acc=SynthNodeInfo())

In [ ]:
for node,data in Gsp.nodes(data=True):
    G.nodes[node]

In [ ]:
Gsp.nodes[20],G.nodes[20]

In [ ]:
Gsp.number_of_nodes(),G.number_of_nodes()

## Community detection to get the clusters

In [ ]:
G,clusters = graph.detect_communities(G)
G.nb_communities = len(clusters)
#c_connectivity = pysad.clusters.cluster_connectivity(G)

In [ ]:
G = graph.remove_small_communities(G,clusters,min_size=10)

In [ ]:
# Save the graph
import networkx as nx
import json
# Save as gexf file
graphfilename = data_path + graphname + '.gexf'
nx.write_gexf(G,graphfilename)
print('Wrote',graphfilename)
